# Density profile $\rho_x(x)$ and surface tension $\gamma_x(x)$

## The goal of this notebook is to analyze a number of configs from a trajectory file and report the density along the $x$-axis. 

For a system that is separated, we should see something like two Heaviside step functions, whereas a mixed system should have two flat, equal, and overlapping densities in that direction.

Pseudocode for this would be something like the following:

```bash
Loop configs:
    loop over particles:
        count[pos[i][x] / dx]++;
```
and then divide by `configs*dx*Ly*Lz`.

My approach:
- [ ] Read in lammps trajectories as pandas DataFrames
- [ ] Make a histogram of $x$-positions for the last $n$ trajectories in a file
- [ ] Rescale the counts appropriately and report with and without type separation

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob

In [ ]:
# density profile by type

pattern = '../equil_traj.dump'
matching_files = glob.glob(pattern)
filename = matching_files[0]
if len(matching_files) >1:
    print(f"Warning: multiple trajectory files found. \nUsing {filename}")
    
num_atoms = 7600
# first, get the boxsize in case we need it:
box = pd.read_csv(filename, nrows = 3, skiprows = 5, header =None, delim_whitespace = True)
# also, get the number of configs:
with open(filename, 'r') as file:
    line_count = file.read().count('\n') + 1
total_configs = line_count // (num_atoms + 9)
names = ["id", "mol", "type", "x", "y", "z"]

In [ ]:
num_configs = total_configs

In [ ]:
# now generate the list of lines to skip:
first_skips = [(num_atoms + 9)*i for i in range(num_configs)]
fs = np.array(first_skips)
skip_arr = np.concatenate([fs + i for i in range(9)])

equil_df = pd.read_csv(filename, delim_whitespace=True, nrows = num_atoms*num_configs, 
                        skiprows= skip_arr, header = None, names= names, index_col=None)
# for equilibration's sake, chop off the first half of the data:
equil_df = equil_df.iloc[len(equil_df)//2:] 

In [ ]:
equil_df

In [ ]:
# density profile:
Lx = box[1][0] - box[0][0]
Ly = box[1][1] - box[0][1]
Lz = box[1][2] - box[0][2]

nbins = 1000

dx = Lx/nbins
counts, bins, patches = plt.hist(equil_df['x'], bins=nbins, color='purple', histtype='step')
counts_1, bins_1, patches_1 = plt.hist(equil_df[equil_df.type == 1]['x'], bins=nbins, color='r', histtype='step')
counts_2, bins_2, patches_2 = plt.hist(equil_df[equil_df.type == 2]['x'], bins=nbins, color='b', histtype='step')
counts/= (num_configs* Ly*Lz * dx/2)
counts_1/= (num_configs* Ly*Lz * dx/2)
counts_2/= (num_configs* Ly*Lz * dx/2)
plt.title("Density Histograms (un-normalized)")
plt.xlabel(r"$x$")
plt.ylabel("Count")
plt.show()
plt.step(bins[:-1], counts, where='mid', color='purple', linestyle='-', linewidth=2)
plt.xlabel("z")
plt.ylabel("Counts/(n*Ly*Lz*dx/2)")
plt.title(r"$\rho_z(z)$")
plt.show()

In [ ]:
fig, axs = plt.subplots(2, 1,  figsize=(6,8), sharex = True)  # 1 row, 2 columns
plt.suptitle(r"Density Profile $\rho_z(z)$")
# Plot for the first subplot
axs[0].step(bins[:-1], counts_1, where='mid', color='r', linestyle='-', linewidth=2)
axs[0].set_title('Type 1')
axs[0].set_ylabel('Counts/(n*Ly*Lz*dx/2)')
axs[0].set_xlabel(r"$x$")

# Plot for the second subplot
axs[1].step(bins[:-1], counts_2, where='mid', color='b', linestyle='-', linewidth=2)
axs[1].set_title('Type 2')
axs[1].set_ylabel('Counts/(n*Ly*Lz*dx/2)')
axs[1].set_xlabel(r"$x$")
plt.show()

## Radial distribution function $g(r)$:

In [ ]:
# import freud
# box_data = box.to_numpy()

In [ ]:
# box = freud.box.Box.from_box(box_data[:, 1] - box_data[:, 0])
# data = equil_df.to_numpy()

# data[..., 3:] -= box.L / 2


# data_1 = equil_df[equil_df.type == 1].to_numpy()
# data_2 = equil_df[equil_df.type == 2].to_numpy()

# data_xyz = data[...,3:].reshape(-1, num_atoms, 3)
# data_xyz_1 = data_1[...,3:].reshape(-1, num_atoms//2, 3)
# data_xyz_2 = data_2[...,3:].reshape(-1, num_atoms//2, 3)

In [ ]:

# rdf = freud.density.RDF(bins=100, r_max=4, r_min=0.0)
# rdf1 = freud.density.RDF(bins=100, r_max=4, r_min=0.0)
# rdf2 = freud.density.RDF(bins=100, r_max=4, r_min=0.0)

# for frame in data_xyz:
#     rdf.compute(system=(box, frame), reset = False)
# # for frame in data_xyz_1:
#     # rdf1.compute(system=(box, frame), reset=False)
# # for frame in data_xyz_2:
#     # rdf2.compute(system=(box, frame), reset=False)

# # Plot the RDF
# plt.plot(rdf.bin_centers, rdf.rdf, label = 'all', c = 'purple')
# # plt.plot(rdf1.bin_centers, rdf1.rdf, label = 'type 1', c = 'r')
# # plt.plot(rdf2.bin_centers, rdf2.rdf, label = 'type 2', c = 'b')
# plt.legend()
# plt.title("Radial Distribution Function")
# plt.xlabel("$r$")
# plt.ylabel("$g(r)$")
# plt.show()


## Plot the surface tension:

Using the exact same method as above, we can get the profile $\gamma(x)$ from the differences between pressure normal to and parallel to the interface:

$$\gamma(x) = \int\limits_{-\infty}^\infty\left( P_{xx} - \frac{1}{2} \left(P_{yy} + P_{zz}\right)\right) dx$$

We just have to read in a different dataset, with each particle's stress components and x-position from each dump. 

In [ ]:
pattern = '../stress.dump'
matching_files = glob.glob(pattern)
filename = matching_files[0]
if len(matching_files) >1:
    print(f"Warning: multiple trajectory files found. \nUsing {filename}")
    
num_atoms = 7600
# first, get the boxsize in case we need it:
box = pd.read_csv(filename, nrows = 3, skiprows = 5, header =None, delim_whitespace = True)
# also, get the number of configs:
with open(filename, 'r') as file:
    line_count = file.read().count('\n') + 1
total_configs = line_count // (num_atoms + 9)


In [ ]:
num_configs = total_configs

In [ ]:
names = ['x', 'y', 'z', 'xx', 'yy', 'zz', 'xy', 'xz', 'yz']

# now generate the list of lines to skip:
first_skips = [(num_atoms + 9)*i for i in range(num_configs)]
fs = np.array(first_skips)
skip_arr = np.concatenate([fs + i for i in range(9)])

In [ ]:
stress_df = pd.read_csv(filename, delim_whitespace=True, nrows = num_atoms*num_configs, 
                        skiprows= skip_arr, header = None, names= names, index_col=None)

# these are pressures, not stresses, so negate them:
components = stress_df.columns.difference(['x'])
stress_df[components] *= -1

# for equilibration's sake, chop off the first half of the data:
stress_df = stress_df.iloc[len(stress_df)//2:] 
stress_df = stress_df.sort_values(by = 'x')

surf_ten = stress_df.xx - 0.5 * (stress_df.yy + stress_df.zz)

In [ ]:
# # this is useful if we wanna shift the boxsize, but still testing this:

# # get the boxsize in the x-direction:
# box_x = box[1][0] - box[0][0]

# stress_df = stress_df.sort_values(by = 'x')
# stress_df.x +=box_x/2 # make it all positive
# stress_df.x = (stress_df.x + box_x/4).mod(box_x) # now shift it over a quarter box

In [ ]:
# this is a the range of surface tension of all particles as a function of x:
# this is averaged over all <config> configurations
plt.scatter(stress_df.x, surf_ten, s = 0.1)
plt.title(f'Distribution of Pressure Differences over {num_configs} configs')
plt.xlabel(r'$x$')
plt.ylabel(r'$P_{xx} - \frac{1}{2} \left(P_{yy} + P_{zz}\right)$')
plt.grid(True)
plt.show()

In [ ]:
# surface tension profile
from scipy import integrate
y_int = integrate.cumulative_trapezoid(surf_ten, stress_df.x, initial=0)

plt.plot(stress_df.x, y_int, c = 'r', linewidth = .5)
plt.xlabel(r'$x$')
plt.ylabel(r'$\gamma(x)$')
plt.title(r'$\int\left[ P_{xx} - \frac{1}{2} \left(P_{yy} + P_{zz}\right)\right] dx$')
plt.grid(True)
plt.show()


In [ ]:
# Assuming stress_df.x contains the x-values and surf_ten contains the y-values
# Calculate the bin edges
nbins = 20
bin_edges = np.linspace(min(stress_df.x), max(stress_df.x), num=nbins+1) # 20 bins
# Find the indices of the bins for each data point
bin_indices = np.digitize(stress_df.x, bin_edges)
# Calculate the mean y-value for each bin
bin_means = [surf_ten[bin_indices == i].mean() for i in range(1, len(bin_edges))]
# Calculate the x-values for the center of each bin
bin_centers = (bin_edges[1:] + bin_edges[:-1]) / 2

# Plot the binned data
plt.plot(bin_centers, bin_means, marker='o', linestyle='-')
plt.xlabel(r'$x$')
plt.ylabel(r'$P_{xx} - \frac{1}{2} \left(P_{yy} + P_{zz}\right)$')
plt.title(f'Pressure Difference (averaged over {nbins} bins, {num_configs} configs)')
plt.grid(True)
plt.show()


In [ ]:
x = bin_centers
y = bin_means
# Perform numerical integration using the trapezoidal rule
y_int = integrate.cumulative_trapezoid(y, x, initial=0)
# Plot the scatter plot
plt.plot(x, y_int, c = 'r', linewidth = 1.5)
# plt.scatter(x, y, c = 'b', s = 2)
plt.xlabel(r'$x$')
plt.ylabel(r'$\gamma(x)$')
plt.title(r'$\int\left[ P_{xx} - \frac{1}{2} \left(P_{yy} + P_{zz}\right)\right] dx$')
plt.grid(True)
plt.show()
